In [182]:
# Imports and Helper Functions
# data Analysis
import pandas as pd
import numpy as np
import random as rng

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last_expr" -- povrat

In [251]:

pd.options.display.float_format = '{:,.2f}'.format
data = pd.read_csv("data.csv")
fighters = pd.read_csv("fighters.csv")
categories_data = pd.read_csv("categories.csv")
fighter_mean = pd.read_csv("fighter_means.csv")
data_edited = pd.read_csv("data_edited.csv")

In [252]:
#data_edited.columns
#fighter_mean.columns
data.columns 
data_edited.columns
fighter_mean.columns
#np.setdiff1d(data_edited.columns, fighter_mean.columns)



Index(['BPrev', 'BStreak', 'B_Age', 'B_Height', 'B_HomeTown', 'B_ID',
       'B_Location', 'B_Name', 'B_Weight',
       'B__Round1_Grappling_Reversals_Landed',
       ...
       'R__Round5_TIP_Ground Time', 'R__Round5_TIP_Guard Control Time',
       'R__Round5_TIP_Half Guard Control Time',
       'R__Round5_TIP_Misc. Ground Control Time',
       'R__Round5_TIP_Mount Control Time', 'R__Round5_TIP_Neutral Time',
       'R__Round5_TIP_Side Control Time', 'R__Round5_TIP_Standing Time',
       'winby', 'winner'],
      dtype='object', length=894)

Index(['B__Prev', 'B__Age', 'B__Height', 'B__Weight',
       'B__Round1_Grappling_Reversals_Landed',
       'B__Round1_Grappling_Standups_Landed',
       'B__Round1_Grappling_Submissions_Attempts',
       'B__Round1_Grappling_Takedowns_Attempts',
       'B__Round1_Grappling_Takedowns_Landed',
       'B__Round1_Strikes_Body Significant Strikes_Attempts',
       ...
       'R__Round5_TIP_Ground Control Time', 'R__Round5_TIP_Ground Time',
       'R__Round5_TIP_Guard Control Time',
       'R__Round5_TIP_Half Guard Control Time',
       'R__Round5_TIP_Misc. Ground Control Time',
       'R__Round5_TIP_Mount Control Time', 'R__Round5_TIP_Neutral Time',
       'R__Round5_TIP_Side Control Time', 'R__Round5_TIP_Standing Time',
       'winner_code'],
      dtype='object', length=879)

Index(['Prev', 'Age', 'Height', 'Weight', 'Round1_Grappling_Reversals_Landed',
       'Round1_Grappling_Standups_Landed',
       'Round1_Grappling_Submissions_Attempts',
       'Round1_Grappling_Takedowns_Attempts',
       'Round1_Grappling_Takedowns_Landed',
       'Round1_Strikes_Body Significant Strikes_Attempts',
       ...
       'Round5_TIP_Ground Control Time', 'Round5_TIP_Ground Time',
       'Round5_TIP_Guard Control Time', 'Round5_TIP_Half Guard Control Time',
       'Round5_TIP_Misc. Ground Control Time', 'Round5_TIP_Mount Control Time',
       'Round5_TIP_Neutral Time', 'Round5_TIP_Side Control Time',
       'Round5_TIP_Standing Time', 'Name'],
      dtype='object', length=440)

In [ ]:
#data.head()
fighters.head()
categories_data.head()
data.tail()

In [ ]:
data.dtypes
data.shape
data.info()

In [ ]:
data.describe(include=['object']) 
#deskriptivan statiskita za brojčane varijable
data.describe()

In [241]:
# Drop rows with NaN values in column 'A'
#data.dropna(subset=['winner'], inplace=True)
#data["winner"].unique()

data = data[(data["winner"] == "red") | (data["winner"] == "blue")]

In [242]:
names = data["B_Name"].unique()
lower_names = [name.lower() for name in names]
np.savetxt('fighters.csv', lower_names, delimiter=',', fmt='%s', header='Fighters', comments='')

In [243]:
#data.fillna(value=0, inplace=True)
data.loc[:, :] = data.fillna(value=0)
data['R_Name'] = data['R_Name'].str.lower()
data['B_Name'] = data['B_Name'].str.lower()

In [244]:
dropdata = data.drop(['Event_ID', 'Fight_ID', 'B_Location', 'B_HomeTown', 'B_ID','B_Name', 'R_Location', 'R_HomeTown','R_ID','R_Name','winby','Date', 'BStreak', 'Last_round', 'Max_round'], axis=1)

dropdata.rename(columns={'BPrev':'B__Prev',
                        'RPrev':'R__Prev',
                        'B_Age':'B__Age',
                        'B_Height':'B__Height',
                        'B_Weight':'B__Weight',
                        'R_Age':'R__Age',
                        'R_Height':'R__Height',
                        'R_Weight':'R__Weight'}, inplace=True)

In [245]:
#pretvaranje object type columns  u kategorijske tip kolumni    
objecttypes = list(dropdata.select_dtypes(include=['O']).columns)
for col in objecttypes:
    dropdata[col] = dropdata[col].astype('category')

cat_columns = dropdata.select_dtypes(['category']).columns
dropdata[cat_columns+"_code"] = dropdata[cat_columns].apply(lambda x: x.cat.codes)

In [246]:
dropdata = dropdata.drop('winner', axis=1)

In [247]:
objecttypes = list(dropdata.select_dtypes(include=['float64']).columns)
for col in objecttypes:
    dropdata[col] = dropdata[col].astype('float32')

dropdata.to_csv('data_edited.csv', index=False)

In [249]:
data_mean = data.drop(['Event_ID', 'Fight_ID', 'B_Location', 'B_HomeTown', 'B_ID', 'R_Location', 'R_HomeTown','R_ID','winby','Date', 'BStreak', 'winner', 'Last_round', 'Max_round'], axis=1)
data_mean.rename(columns={'BPrev':'B__Prev',
                        'RPrev':'R__Prev',
                        'B_Age':'B__Age',
                        'B_Height':'B__Height',
                        'B_Weight':'B__Weight',
                        'R_Age':'R__Age',
                        'R_Height':'R__Height',
                        'R_Weight':'R__Weight',
                        'B_Name': 'B__Name',
                        'R_Name': 'R__Name'}, inplace=True)

combined_all = pd.DataFrame()

for i, name in enumerate(fighters['Fighters']):
    conor = data_mean.loc[ (data_mean['B__Name'] == name)]
    filtered_columns = conor.filter(regex='^R', axis=1)
    conorB = conor.drop(filtered_columns.columns, axis=1)

    new_columns = {col: col.replace("B__", "") for col in conorB.columns if col.startswith("B__")}
    conorB = conorB.rename(columns=new_columns)
    conorB = conorB.drop("Name", axis=1)

    conor = data_mean.loc[ (data_mean['R__Name'] == name)]
    filtered_columns = conor.filter(regex='^B', axis=1)
    conorR = conor.drop(filtered_columns.columns, axis=1)

    new_columns = {col: col.replace("R__", "") for col in conorR.columns if col.startswith("R__")}
    conorR = conorR.rename(columns=new_columns)
    conorR = conorR.drop("Name", axis=1)
    # conorR['Last_round'] = conorR.pop('Last_round')
    # conorR['Max_round'] = conorR.pop('Max_round')
    combined_df = pd.concat([conorR, conorB], ignore_index=True)
    combined_df = combined_df.mean().to_frame().transpose()

    combined_df['Name'] = name
    combined_all = pd.concat([combined_all, combined_df], ignore_index=True)


In [250]:
objecttypes = list(combined_all.select_dtypes(include=['float64']).columns)
for col in objecttypes:
    combined_all[col] = combined_all[col].astype('float32')

combined_all.to_csv('fighter_means.csv', index=False, header=True)


In [ ]:
#for feature in data.dtypes[data.dtypes == 'object'].index: 
df = {'Name': data['B_Location'].value_counts().head(15).index, 'Count': data['B_Location'].value_counts().head(15).values}
df = pd.DataFrame(df)
sns.countplot(x='winby', data=data) 
plt.show() 

In [ ]:
df = {  'Name': data['B_Location'].value_counts().head(15).index, 
        'Count': data['B_Location'].value_counts().head(15).values}
df = pd.DataFrame(df)
sns.barplot(data=df, y='Name', x='Count')
plt.show() 

In [ ]:
for feature in data.dtypes[data.dtypes == 'object'].index: 
    sns.countplot(y=feature, data=dropdata) 
    plt.show() 

In [ ]:
sns.countplot(y='winner_code', data=dropdata) 
plt.show() 


In [ ]:
#for feature in data.dtypes[data.dtypes == 'object'].index: 
sns.countplot(y='R__Weight', data=dropdata) 
sns.countplot(y='B__Weight', data=dropdata) 
plt.show() 

In [ ]:
#Basic Correlation Matrix
corrmat = dropdata.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
# Subset Correlation Matrix
k = 10 #number of variables for heatmap
corrmat = dropdata.corr()
cols = corrmat.nlargest(k, 'winner')['winner'].index
cm = np.corrcoef(dropdata[cols].values.T)
sns.set(font_scale=1)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 5}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
Gradient Boosting Classifier was 
applied with a multinomial deviance loss 
function and a learning rate of 0.01, 
producing the following relative variable 
importances:

In [ ]:
sns.lmplot(x="B__Round3_Strikes_Body Significant Strikes_Attempts", 
            y="B__Round3_Strikes_Body Significant Strikes_Landed", 
            col="winner", hue="winner", data=dropdata, col_wrap=2)